In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
from pathlib import Path
from typing import *
import torch
import torch.optim as optim
import numpy as np
import pandas as pd
from functools import partial
from overrides import overrides

from allennlp.data import Instance
from allennlp.data.token_indexers import TokenIndexer
from allennlp.data.tokenizers import Token
from allennlp.nn import util as nn_util

In [3]:
class Config(dict):
    def __init__(self, **kwargs):
        super().__init__(**kwargs)
        for k, v in kwargs.items():
            setattr(self, k, v)
    
    def set(self, key, val):
        self[key] = val
        setattr(self, key, val)
        
config = Config(
    testing=True,
    seed=1,
    batch_size=32,
    lr=3e-4,
    epochs=2,
    hidden_sz=64,
    max_seq_len=100, # necessary to limit memory usage
    max_vocab_size=100000,
)

In [4]:
from allennlp.common.checks import ConfigurationError

In [5]:
USE_GPU = torch.cuda.is_available()

In [6]:
DATA_ROOT = Path("../data") / "jigsaw"

Set random seed manually to replicate results

In [7]:
torch.manual_seed(config.seed)

# Load Data

In [8]:
from allennlp.data.vocabulary import Vocabulary
from allennlp.data.dataset_readers import DatasetReader

### Prepare dataset

In [9]:
label_cols = ["toxic", "severe_toxic", "obscene",
              "threat", "insult", "identity_hate"]

In [10]:
from allennlp.data.fields import TextField, MetadataField, ArrayField

class JigsawDatasetReader(DatasetReader):
    def __init__(self, tokenizer: Callable[[str], List[str]]=lambda x: x.split(),
                 token_indexers: Dict[str, TokenIndexer] = None,
                 max_seq_len: Optional[int]=config.max_seq_len) -> None:
        super().__init__(lazy=False)
        self.tokenizer = tokenizer
        self.token_indexers = token_indexers or {"tokens": SingleIdTokenIndexer()}
        self.max_seq_len = max_seq_len

    @overrides
    def text_to_instance(self, tokens: List[Token], id: str=None,
                         labels: np.ndarray=None) -> Instance:
        sentence_field = TextField(tokens, self.token_indexers)
        fields = {"tokens": sentence_field}
        
        id_field = MetadataField(id)
        fields["id"] = id_field
        
        if labels is None:
            labels = np.zeros(len(label_cols))
        label_field = ArrayField(array=labels)
        fields["label"] = label_field

        return Instance(fields)
    
    @overrides
    def _read(self, file_path: str) -> Iterator[Instance]:
        df = pd.read_csv(file_path)
        if config.testing: df = df.head(1000)
        for i, row in df.iterrows():
            yield self.text_to_instance(
                [Token(x) for x in self.tokenizer(row["comment_text"])],
                row["id"], row[label_cols].values,
            )

### Prepare token handlers

We will use the spacy tokenizer here

In [94]:
from allennlp.data.tokenizers.word_splitter import SpacyWordSplitter
from allennlp.data.token_indexers import SingleIdTokenIndexer

# the token indexer is responsible for mapping tokens to integers
token_indexer = SingleIdTokenIndexer()

def tokenizer(x: str):
    return [w.text for w in
            SpacyWordSplitter(language='en_core_web_sm', 
                              pos_tags=False).split_words(x)[:config.max_seq_len]]

In [95]:
reader = JigsawDatasetReader(
    tokenizer=tokenizer,
    token_indexers={"tokens": token_indexer}
)

In [96]:
train_ds, test_ds = (reader.read(DATA_ROOT / fname) for fname in ["train.csv", "test_proced.csv"])
val_ds = None

267it [00:00, 1252.37it/s]
251it [00:00, 1291.15it/s]


In [97]:
len(train_ds)

267

In [98]:
train_ds[:10]

In [99]:
vars(train_ds[0].fields["tokens"])

{'tokens': [Explanation,
  Why,
  the,
  edits,
  made,
  under,
  my,
  username,
  Hardcore,
  Metallica,
  Fan,
  were,
  reverted,
  ?,
  They,
  were,
  n't,
  vandalisms,
  ,,
  just,
  closure,
  on,
  some,
  GAs,
  after,
  I,
  voted,
  at,
  New,
  York,
  Dolls,
  FAC,
  .,
  And,
  please,
  do,
  n't,
  remove,
  the,
  template,
  from,
  the,
  talk,
  page,
  since,
  I,
  'm,
  retired,
  now.89.205.38.27],
 '_token_indexers': {'tokens': <allennlp.data.token_indexers.single_id_token_indexer.SingleIdTokenIndexer at 0x7efd286ce128>},
 '_indexed_tokens': None,
 '_indexer_name_to_indexed_token': None,
 '_token_index_to_indexer_name': None}

### Prepare vocabulary

In [100]:
vocab = Vocabulary.from_instances(train_ds, max_vocab_size=config.max_vocab_size)

100%|██████████| 267/267 [00:00<00:00, 39300.90it/s]


### Prepare iterator

The iterator is responsible for batching the data and preparing it for input into the model. We'll use the BucketIterator that batches text sequences of smilar lengths together.

In [101]:
from allennlp.data.iterators import BucketIterator

In [102]:
iterator = BucketIterator(batch_size=config.batch_size, 
                          sorting_keys=[("tokens", "num_tokens")],
                         )

We need to tell the iterator how to numericalize the text data. We do this by passing the vocabulary to the iterator. This step is easy to forget so be careful! 

In [103]:
iterator.index_with(vocab)

### Read sample

In [104]:
batch = next(iter(iterator(train_ds)))

In [105]:
batch

{'tokens': {'tokens': tensor([[   5,   23,   23,  ...,    3,  805, 1588],
          [   5,  870,   10,  ...,   57,  347,   35],
          [   5,  103,    5,  ..., 2925,    3,   11],
          ...,
          [   5, 1309,   22,  ...,   57,  141,   31],
          [   7,   25,  392,  ...,   73, 1407,   23],
          [   5, 1589,    3,  ...,    5,   16,  810]])},
 'id': ['001d874a4d3e8813',
  '00328eadb85b3010',
  '00822d0d01752c7e',
  '004b073d5b456b15',
  '008a1e9c45de8138',
  '00584d887401f47b',
  '00037261f536c51d',
  '00905910dcbcc8aa',
  '003a19c04c079bf7',
  '005e6b1369cbe377',
  '0038d1dc2ad29469',
  '0001b41b1c6bb37e',
  '0034065c7b12a7a2',
  '0033b9d5ccd499fb',
  '0091aec11b57d12e',
  '003b9f448ee4a29d',
  '002746baedcdff10',
  '00a317acddff8a62',
  '00a7d82371e10c5d',
  '00151a9f93c6b059',
  '0048de0c9422f64f',
  '0006f16e4e9f292e',
  '004b975fabbbffa9',
  '0021fe88bc4da3e6',
  '00882ab8cfa42274',
  '0063a8786a7034fc',
  '006774d59329b7bd',
  '00218d74784ce50b',
  '007e1e47cd0e2

In [106]:
batch["tokens"]["tokens"]

tensor([[   5,   23,   23,  ...,    3,  805, 1588],
        [   5,  870,   10,  ...,   57,  347,   35],
        [   5,  103,    5,  ..., 2925,    3,   11],
        ...,
        [   5, 1309,   22,  ...,   57,  141,   31],
        [   7,   25,  392,  ...,   73, 1407,   23],
        [   5, 1589,    3,  ...,    5,   16,  810]])

In [61]:
batch["tokens"]["tokens"].shape

torch.Size([32, 19])

# Prepare Model

In [62]:
import torch
import torch.nn as nn
import torch.optim as optim

In [63]:
from allennlp.modules.seq2vec_encoders import Seq2VecEncoder, PytorchSeq2VecWrapper
from allennlp.nn.util import get_text_field_mask
from allennlp.models import Model
from allennlp.modules.text_field_embedders import TextFieldEmbedder

class BaselineModel(Model):
    def __init__(self, word_embeddings: TextFieldEmbedder,
                 encoder: Seq2VecEncoder,
                 out_sz: int=len(label_cols)):
        super().__init__(vocab)
        self.word_embeddings = word_embeddings
        self.encoder = encoder
        self.projection = nn.Linear(self.encoder.get_output_dim(), out_sz)
        self.loss = nn.BCEWithLogitsLoss()
        
    def forward(self, tokens: Dict[str, torch.Tensor],
                id: Any, label: torch.Tensor) -> torch.Tensor:
        mask = get_text_field_mask(tokens)
        embeddings = self.word_embeddings(tokens)
        state = self.encoder(embeddings, mask)
        class_logits = self.projection(state)
        
        output = {"class_logits": class_logits}
        output["loss"] = self.loss(class_logits, label)

        return output

### Prepare embeddings

In [64]:
from allennlp.modules.token_embedders import Embedding
from allennlp.modules.text_field_embedders import BasicTextFieldEmbedder

token_embedding = Embedding(num_embeddings=config.max_vocab_size + 2,
                            embedding_dim=300, padding_index=0)
# the embedder maps the input tokens to the appropriate embedding matrix
word_embeddings: TextFieldEmbedder = BasicTextFieldEmbedder({"tokens": token_embedding})

In [65]:
from allennlp.modules.seq2vec_encoders import PytorchSeq2VecWrapper
encoder: Seq2VecEncoder = PytorchSeq2VecWrapper(nn.LSTM(word_embeddings.get_output_dim(),
                                                        config.hidden_sz, bidirectional=True, batch_first=True))

Notice how simple and modular the code for initializing the model is. All the complexity is delegated to each component.

In [66]:
model = BaselineModel(
    word_embeddings, 
    encoder, 
)

In [67]:
if USE_GPU: model.cuda()
else: model
#device = torch.device('cpu')
#model = model.to(device)

# Basic sanity checks

In [68]:
batch = nn_util.move_to_device(batch, 0 if USE_GPU else -1)

In [69]:
tokens = batch["tokens"]
labels = batch

In [70]:
tokens

{'tokens': tensor([[2812,   11, 2813,   42,  205,  272, 1054,   73, 2814,  604,    3,  817,
             0,    0,    0,    0,    0,    0,    0],
         [1219,   21,    8,  259,   11,  469,  470,   14,   24,  375,  311,    4,
          1220,   29,    0,    0,    0,    0,    0],
         [2729,   32, 2730,   23, 2731, 2732, 2733, 2734,    2, 2735,   32,   49,
            31,    5,    0,    0,    0,    0,    0],
         [ 859,   13,    4, 2653,   12,    4,  151,  217,    6, 2654,    2,   74,
            34,    0,    0,    0,    0,    0,    0],
         [  42, 2479, 1028,   72, 2480,    2,   42, 2481, 1028,   89,   16,    2,
             0,    0,    0,    0,    0,    0,    0],
         [ 240,   79,  169,   25,  209,  936,   27,   62,  191,   41,   94,   82,
            23,    0,    0,    0,    0,    0,    0],
         [1281,   34,  158,   30,  123,    3,  318,   35,   75,   10, 1282,  178,
            34, 1283,    2,    0,    0,    0,    0],
         [ 651,   22,  322,  852,  206,    3,

In [71]:
mask = get_text_field_mask(tokens)
mask

tensor([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0],
        [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0],
        [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0],
        [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0],
        [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0],
        [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0],
        [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0],
        [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0],
        [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0],
        [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0],
        [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0],
        [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0],
        [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0],
        [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0],
        [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0,

In [72]:
embeddings = model.word_embeddings(tokens)
state = model.encoder(embeddings, mask)
class_logits = model.projection(state)
class_logits

tensor([[-0.0388,  0.0792, -0.0304,  0.0286,  0.0153, -0.0261],
        [-0.0384,  0.0808, -0.0330,  0.0290,  0.0156, -0.0258],
        [-0.0397,  0.0801, -0.0309,  0.0281,  0.0164, -0.0257],
        [-0.0392,  0.0812, -0.0321,  0.0293,  0.0146, -0.0260],
        [-0.0387,  0.0804, -0.0316,  0.0303,  0.0162, -0.0256],
        [-0.0391,  0.0807, -0.0318,  0.0291,  0.0183, -0.0262],
        [-0.0398,  0.0803, -0.0328,  0.0282,  0.0166, -0.0253],
        [-0.0377,  0.0797, -0.0305,  0.0277,  0.0168, -0.0246],
        [-0.0405,  0.0805, -0.0317,  0.0289,  0.0170, -0.0241],
        [-0.0403,  0.0815, -0.0304,  0.0279,  0.0166, -0.0264],
        [-0.0370,  0.0800, -0.0305,  0.0282,  0.0159, -0.0240],
        [-0.0387,  0.0817, -0.0302,  0.0292,  0.0162, -0.0279],
        [-0.0375,  0.0825, -0.0316,  0.0292,  0.0156, -0.0270],
        [-0.0388,  0.0807, -0.0306,  0.0266,  0.0170, -0.0245],
        [-0.0390,  0.0805, -0.0314,  0.0264,  0.0167, -0.0253],
        [-0.0398,  0.0801, -0.0304,  0.0

In [73]:
model(**batch)

{'class_logits': tensor([[-0.0388,  0.0792, -0.0304,  0.0286,  0.0153, -0.0261],
         [-0.0384,  0.0808, -0.0330,  0.0290,  0.0156, -0.0258],
         [-0.0397,  0.0801, -0.0309,  0.0281,  0.0164, -0.0257],
         [-0.0392,  0.0812, -0.0321,  0.0293,  0.0146, -0.0260],
         [-0.0387,  0.0804, -0.0316,  0.0303,  0.0162, -0.0256],
         [-0.0391,  0.0807, -0.0318,  0.0291,  0.0183, -0.0262],
         [-0.0398,  0.0803, -0.0328,  0.0282,  0.0166, -0.0253],
         [-0.0377,  0.0797, -0.0305,  0.0277,  0.0168, -0.0246],
         [-0.0405,  0.0805, -0.0317,  0.0289,  0.0170, -0.0241],
         [-0.0403,  0.0815, -0.0304,  0.0279,  0.0166, -0.0264],
         [-0.0370,  0.0800, -0.0305,  0.0282,  0.0159, -0.0240],
         [-0.0387,  0.0817, -0.0302,  0.0292,  0.0162, -0.0279],
         [-0.0375,  0.0825, -0.0316,  0.0292,  0.0156, -0.0270],
         [-0.0388,  0.0807, -0.0306,  0.0266,  0.0170, -0.0245],
         [-0.0390,  0.0805, -0.0314,  0.0264,  0.0167, -0.0253],
         

In [74]:
loss = model(**batch)["loss"]

In [75]:
loss

tensor(0.6969, grad_fn=<BinaryCrossEntropyWithLogitsBackward>)

In [76]:
loss.backward()

# Train

In [77]:
optimizer = optim.Adam(model.parameters(), lr=config.lr)

In [78]:
from allennlp.training.trainer import Trainer

trainer = Trainer(
    model=model,
    optimizer=optimizer,
    iterator=iterator,
    train_dataset=train_ds,
    cuda_device=0 if USE_GPU else -1,
    num_epochs=config.epochs,
)

In [79]:
metrics = trainer.train()

loss: 0.6888 ||: 100%|██████████| 9/9 [00:01<00:00,  5.79it/s]
loss: 0.6701 ||: 100%|██████████| 9/9 [00:01<00:00,  5.95it/s]


# Generating Predictions

AllenNLP is slightly lacking in its ability to convert datasets to predictions (though it has extensive support for converting single examples to predictions). Therefore, we'll write our own Predictor class to handle this job for us.

Thankfully, a lot of the tools we used eariler can easily be extended to prediction. Here's how.

In [80]:
from allennlp.data.iterators import DataIterator
from tqdm import tqdm
from scipy.special import expit # the sigmoid function

def tonp(tsr): return tsr.detach().cpu().numpy()

class Predictor:
    def __init__(self, model: Model, iterator: DataIterator,
                 cuda_device: int=-1) -> None:
        self.model = model
        self.iterator = iterator
        self.cuda_device = cuda_device
        
    def _extract_data(self, batch) -> np.ndarray:
        out_dict = self.model(**batch)
        return expit(tonp(out_dict["class_logits"]))
    
    def predict(self, ds: Iterable[Instance]) -> np.ndarray:
        pred_generator = self.iterator(ds, num_epochs=1, shuffle=False)
        self.model.eval()
        pred_generator_tqdm = tqdm(pred_generator,
                                   total=self.iterator.get_num_batches(ds))
        preds = []
        with torch.no_grad():
            for batch in pred_generator_tqdm:
                batch = nn_util.move_to_device(batch, self.cuda_device)
                preds.append(self._extract_data(batch))
        return np.concatenate(preds, axis=0)

We'll need a iterator that goes sequentially over our data. We'll use the 

In [81]:
from allennlp.data.iterators import BasicIterator
# iterate over the dataset without changing its order
seq_iterator = BasicIterator(batch_size=64)
seq_iterator.index_with(vocab)

In [82]:
predictor = Predictor(model, seq_iterator, cuda_device=0 if USE_GPU else -1)
train_preds = predictor.predict(train_ds) 
test_preds = predictor.predict(test_ds) 

100%|██████████| 4/4 [00:00<00:00, 37.44it/s]


# A Final Note on Predictors

AllenNLP also provides predictors that take strings as input and outputs model predictions. They're handy if you want to create simple demo or need to make predictions on entirely new data, but since we've already read data as datasets and want to preserve their order, we didn't use them above. 

In [83]:
from allennlp.predictors.sentence_tagger import SentenceTaggerPredictor

In [84]:
tagger = SentenceTaggerPredictor(model, reader)

In [85]:
tagger.predict("this tutorial was great!")

{'class_logits': [-0.10390057414770126,
  -0.004794597625732422,
  -0.10375700891017914,
  -0.031739577651023865,
  -0.06213139370083809,
  -0.10052867233753204],
 'loss': 0.6600041389465332}

In [86]:
tagger.predict("this tutorial was horrible!")

{'class_logits': [-0.10390057414770126,
  -0.004794597625732422,
  -0.10375700891017914,
  -0.031739577651023865,
  -0.06213139370083809,
  -0.10052867233753204],
 'loss': 0.6600041389465332}

In [87]:
test_preds

array([[0.4723574 , 0.49640787, 0.469895  , 0.48834574, 0.48394322,
        0.47366717],
       [0.4724129 , 0.49696982, 0.4707191 , 0.4893371 , 0.48351476,
        0.4739319 ],
       [0.47110337, 0.49457818, 0.46839222, 0.487233  , 0.48218426,
        0.4726479 ],
       ...,
       [0.47228163, 0.4954556 , 0.46950588, 0.48767394, 0.48294523,
        0.47296512],
       [0.47272792, 0.49655986, 0.47035402, 0.48888558, 0.4842466 ,
        0.47403792],
       [0.47122023, 0.4948841 , 0.468895  , 0.48726082, 0.48247686,
        0.47307032]], dtype=float32)